# 掌握LoRA参数高效方法适配垂直领域

## 1. 什么是大模型微调？有什么作用？有哪些分类？

### 什么是大模型微调？

**大模型微调（Fine-tuning）**是指在预训练好的大型语言模型（LLM）的基础上，使用特定任务的数据集进行进一步训练的过程。这些预训练模型通常在海量的通用文本数据上进行过训练，拥有强大的语言理解和生成能力。然而，它们可能不擅长处理特定领域、特定格式或特定风格的任务。微调的目的就是让这些通用模型更好地适应特定场景的需求。

### 有什么作用？

大模型微调的主要作用包括：

* **提升模型在特定任务上的性能**：通过在少量特定数据上进行训练，模型能够学习到该任务的细节和模式，从而显著提高在问答、摘要、情感分析、代码生成等具体任务上的准确性和效果。
* **适应特定领域知识**：通用大模型可能不了解某些专业领域的术语、概念或业务流程。微调可以让模型掌握这些特定领域的知识，例如医疗、法律、金融或编程领域的知识。
* **降低训练成本**：从头开始训练一个大型模型需要巨大的计算资源和数据。微调则可以在预训练模型的基础上进行，只需要相对较少的数据和计算资源，大大降低了训练门槛和成本。
* **快速迭代和部署**：通过微调，可以快速地根据业务需求调整模型，并将其部署到实际应用中，加速产品开发周期。
* **优化模型行为和风格**：微调可以使模型输出符合特定的语气、风格或格式要求，例如让模型回答得更简洁、更友好、或符合某个品牌形象。

### 有哪些分类？

大模型微调的方法有很多种，可以根据训练的参数量、训练方式等进行分类：

* **全量微调（Full Fine-tuning）**：
    * **特点**：更新模型所有的参数。
    * **优点**：理论上能达到最佳性能，模型适应性最强。
    * **缺点**：计算资源和存储需求巨大，容易过拟合。对于参数量庞大的大模型来说，成本极高且不切实际。

* **参数高效微调（Parameter-Efficient Fine-Tuning, PEFT）**：
    * **特点**：只更新模型参数的一小部分，或者引入少量额外参数进行训练。这是目前主流的大模型微调方法。
    * **优点**：显著降低计算和存储成本，避免灾难性遗忘，降低过拟合风险。
    * **缺点**：在某些极端情况下，性能可能略低于全量微调，但通常可以忽略不计。
    * **常见方法**：
        * **LoRA (Low-Rank Adaptation)**：通过在预训练模型的特定层注入可训练的低秩矩阵来更新模型。
        * **Prefix Tuning / P-Tuning v2**：在输入序列前添加可学习的前缀或前缀向量，只训练这些前缀参数。
        * **Adapter Tuning**：在预训练模型的每一层中插入小型、可训练的神经网络模块（Adapter）。
        * **BitFit**：只微调模型中的Bias参数。
        * **QLoRA (Quantized LoRA)**：在4比特量化模型的基础上进行LoRA微调，进一步降低内存占用。

* **指令微调（Instruction Tuning）**：
    * **特点**：将任务转化为指令形式，训练模型理解并遵循指令，从而提高模型的泛化能力。
    * **作用**：使模型能够更好地响应用户的自然语言指令，是提升模型“可控性”和“泛化性”的关键技术。

* **RLHF（Reinforcement Learning from Human Feedback）**：
    * **特点**：结合人类反馈的强化学习，通过人类对模型输出的偏好排序来训练一个奖励模型，然后使用这个奖励模型进一步微调语言模型。
    * **作用**：使模型输出与人类价值观和偏好对齐，减少有害、偏见或不准确的回答，提高模型的用户体验。ChatGPT等流行模型都广泛采用了RLHF。

---

## 2. LoRA 是一种什么微调，特点是什么？

### 什么是LoRA？

**LoRA (Low-Rank Adaptation of Large Language Models)** 是一种高效的**参数高效微调（PEFT）**方法。它的核心思想是：在预训练模型的大部分权重保持不变的情况下，通过在模型中添加少量的、可训练的**低秩（low-rank）矩阵**来适应特定任务。

具体来说，对于预训练模型中的一个权重矩阵 $W_0$，LoRA 不直接修改 $W_0$，而是在其旁边引入两个小的矩阵 $A$ 和 $B$。在训练过程中，只有这两个低秩矩阵 $A$ 和 $B$ 被更新，而原始的 $W_0$ 保持冻结。在推理时，这些训练好的低秩矩阵可以与 $W_0$ 相加（$W_0 + BA$），从而形成一个适应了新任务的新权重矩阵。由于 $A$ 和 $B$ 的维度远小于 $W_0$，因此需要训练的参数量大大减少。

### LoRA的特点：

* **参数量极小**：这是LoRA最显著的特点。它只引入并训练极少量的参数（通常是原始模型参数的0.01% - 1%），极大地降低了训练的计算和存储成本。
* **训练速度快**：由于只更新少量参数，训练速度比全量微调快得多。
* **内存占用低**：在微调过程中，由于大部分原始模型参数是冻结的，GPU内存占用显著降低，使得在消费级GPU上微调大型模型成为可能。
* **避免灾难性遗忘**：由于原始模型的知识被保留，LoRA在适应新任务的同时，能够有效避免对预训练模型通用能力的损害（即灾难性遗忘）。
* **模块化和可插拔**：训练好的LoRA权重（即矩阵A和B）非常小，可以作为独立的适配器存储和加载。这意味着您可以为同一个基础模型训练多个不同的LoRA适配器，每个适配器对应一个特定任务，并在需要时进行切换，非常灵活。
* **推理高效**：在推理时，可以将LoRA权重与原始模型权重合并，无需额外的推理延迟。
* **多任务支持**：可以针对不同任务训练独立的LoRA模块，然后在推理时根据任务切换对应的LoRA模块。

---

## 3. 有哪些常用的微调框架，SWIFT框架的优势？

### 常用的微调框架：

目前市面上有许多用于大模型微调的框架，它们各有特点，旨在简化微调流程，提高效率：

* **Hugging Face Transformers**：
    * **特点**：最流行和广泛使用的NLP库之一，提供了大量预训练模型和易于使用的API，支持各种模型结构和微调方法。
    * **优势**：社区活跃，模型资源丰富，兼容性好。
    * **缺点**：对于复杂的PEFT方法，可能需要手动组合不同模块。

* **PEFT (Parameter-Efficient Fine-tuning)**：
    * **特点**：Hugging Face团队推出的专门用于参数高效微调的库，集成了多种PEFT方法（如LoRA, Prefix Tuning, P-Tuning v2, AdaLoRA等）。
    * **优势**：专注于PEFT，提供统一的API，易于使用和切换不同的PEFT方法。
    * **缺点**：通常与Transformers库结合使用。


### SWIFT框架的优势：

**SWIFT (Scalable, Writable, Intelligent, Fast, Tunable)** 是ModelScope社区推出的一个专注于大模型训练和推理的框架。它在微调方面具有以下显著优势：

* **一站式解决方案**：SWIFT提供了一个从数据处理、模型加载、微调到评估和部署的完整流程。用户无需手动处理复杂的代码逻辑，只需通过简单的配置即可完成微调。
* **高度集成ModelScope生态**：作为ModelScope的官方工具，SWIFT与ModelScope上的海量模型和数据集无缝衔接。用户可以直接通过模型ID和数据集ID加载资源，极大地简化了准备工作。
* **对PEFT方法的全面支持**：SWIFT内置了对多种主流PEFT方法（包括LoRA、QLoRA、LoRA+、AdaLoRA、NEFTune等）的封装，用户可以轻松切换不同的微调策略，并灵活配置相关参数。
* **命令行接口（CLI）与Python API并存**：SWIFT提供了简洁的命令行工具，用户可以通过一行命令快速启动微调；同时，也提供了Python API，方便开发者进行更灵活的编程控制和自定义。
* **优化和加速**：SWIFT在底层集成了多种优化技术，如混合精度训练、梯度累积、高效的数据加载等，以确保训练过程高效且稳定。
* **用户友好性**：SWIFT的设计哲学是“开箱即用”，它抽象了底层复杂性，让即使是不熟悉大模型训练细节的用户也能快速上手。例如，对于自我认知数据集，它直接提供了 `model_name` 和 `model_author` 参数，简化了数据替换逻辑。
* **持续更新与维护**：作为ModelScope的核心项目，SWIFT会持续更新，支持最新的模型架构和微调技术，并提供社区支持。

SWIFT框架特别适合希望快速、高效地进行模型微调的开发者和研究者。



## 4\. SWIFT框架微调Qwen2.5-0.5B模型实现自我认知功能

在之前的讨论中，我们了解了如何使用SWIFT框架对Qwen2.5-0.5B模型进行微调，并加载了`self-cognition`数据集。现在，让我们深入探讨一下这里所说的“实现自我认知功能”具体指什么，以及它是如何通过微调实现的。

### 4.1 什么是实现自我认知功能

**大模型的“自我认知”（Self-Cognition）功能，是指让语言模型能够识别、理解并准确回答与自身相关信息的能力。** 简单来说，就是当用户询问关于模型本身的问题时，比如：“你叫什么名字？”、“谁开发了你？”、“你是一个AI模型吗？”，模型能够给出预设的、准确的答案，而不是生成随机的、不相关或错误的回复。

这种能力对于提升用户体验和模型的可靠性至关重要。一个具备自我认知能力的模型，会显得更加智能、专业和可信。它能更好地扮演一个“角色”，例如一个智能助手或客服机器人，避免产生混淆或误导用户的回答。

通过对Qwen2.5-0.5B模型进行微调，并利用特定的“自我认知”数据集，我们就是在教导这个模型如何正确地“认识自己”，并以设定的方式进行自我介绍。

### 4.2 环境准备

为了顺利进行Qwen2.5-0.5B模型的微调，您需要先准备好相应的运行环境。

1.  **Python 环境**: 建议使用 Python 3.8 或更高版本。

2.  **GPU 环境**: 由于大模型训练对计算资源要求较高，您需要一块具备足够显存的NVIDIA GPU。推荐使用显存至少为 8GB 的 GPU，如 RTX 3060/4060 或更高级别显卡。确保您的系统已正确安装NVIDIA驱动、CUDA Toolkit 和 cuDNN。

3.  **安装 SWIFT 框架**: 这是最核心的步骤。SWIFT 框架集成了大模型微调所需的各种工具和库。您可以通过以下命令安装：

    ```bash
    pip install 'ms-swift[llm]' -U
    ```

    `[llm]` 表示安装与大型语言模型相关的组件和依赖，`-U` 表示升级到最新版本。

-----

## 5\. 代码示例

### 5.1 数据集

我们使用的微调数据集是 **ModelScope 上的 `swift/self-cognition` 数据集**。

  * **数据集来源**: 该数据集托管在 ModelScope 社区，专门为训练语言模型的自我认知能力而设计。
  * **数据集内容**: 它包含一系列与模型自身相关的问答对，例如：
      * “你叫什么名字？” -\> “我叫`{{NAME}}`。”
      * “谁创造了你？” -\> “我由`{{AUTHOR}}`团队开发。”
      * “你是一个人类吗？” -\> “不，我是一个人工智能助手。”
      * 等等。
  * **占位符**: 数据集中使用了 `{{NAME}}` 和 `{{AUTHOR}}` 等占位符。在微调过程中，SWIFT 框架允许您通过 `model_name` 和 `model_author` 参数来替换这些占位符，从而让模型学习到您设定的具体名称和开发者信息。这种设计使得数据集具有通用性，可以应用于不同的模型实例。
  * **数据量**: 该数据集的数据量通常不大（例如，原始版本包含 108 条数据），这对于 LoRA 这种高效微调方法来说已经足够。

### 5.2 代码解释

```python
#!/bin/bash

# 这个脚本用于使用LoRA方法微调Qwen3-0.6B模型
# LoRA是一种参数高效微调技术，通过添加小型适配器来调整预训练模型，而不需要更新所有参数

# 指定使用的GPU设备，这里设置为使用第一块GPU
CUDA_VISIBLE_DEVICES=0 swift sft \
    --model Qwen/Qwen3-0.6B \
    --train_type lora \
    --dataset 'swift/Qwen3-SFT-Mixin#2000' 'swift/self-cognition:empty_think#600' \
    --torch_dtype bfloat16 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32 \
    --target_modules all-linear \
    --gradient_accumulation_steps 16 \
    --eval_steps 50 \
    --save_steps 50 \
    --save_total_limit 2 \
    --logging_steps 5 \
    --max_length 2048 \
    --output_dir output \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 4 \
    --use_liger_kernel true \
    --load_from_cache_file false \
    --loss_scale ignore_empty_think \
    --model_author swift \
    --model_name swift-robot

# 参数详细说明：
#
# --model Qwen/Qwen3-0.6B
#   指定基础模型，使用ModelScope上的Qwen3-0.6B模型，这是一个轻量级但功能强大的基础模型
#
# --train_type lora
#   使用LoRA(低秩适配器)微调方法，可以大幅减少训练参数量和显存占用
#   相比全参数微调，LoRA只训练很小一部分参数，效率更高
#
# --dataset 'swift/Qwen3-SFT-Mixin#2000' 'swift/self-cognition:empty_think#600'
#   指定训练数据集：
#   - 'swift/Qwen3-SFT-Mixin#2000'：使用2000条通用指令微调数据，提高模型的基础能力
#   - 'swift/self-cognition:empty_think#600'：使用600条自我认知数据，让模型了解自己的身份
#     empty_think表示忽略思考过程，只关注最终回答
#
# --torch_dtype bfloat16
#   使用bfloat16精度进行训练，相比float32可以节省显存并加速训练，同时保持数值稳定性
#
# --num_train_epochs 1
#   训练轮次设置为1轮，对于LoRA微调和高质量数据集，通常1轮就能达到不错的效果
#
# --per_device_train_batch_size 1
#   每个设备(GPU)上的训练批次大小为1，小批量可以减少显存占用
#
# --per_device_eval_batch_size 1
#   每个设备上的评估批次大小为1
#
# --learning_rate 1e-4
#   设置学习率为1e-4，这是LoRA微调的推荐学习率，平衡了训练速度和稳定性
#
# --lora_rank 8
#   LoRA的秩为8，决定了LoRA适配器的参数量和表达能力
#   较小的秩(如4)参数更少但能力有限，较大的秩(如16)能力更强但参数更多
#
# --lora_alpha 32
#   LoRA的缩放参数为32，控制LoRA更新的影响程度
#   通常设置为lora_rank的4倍左右，可以增强训练稳定性
#
# --target_modules all-linear
#   将LoRA应用到所有线性层，使模型能更全面地学习
#   对于自我认知训练特别重要，因为身份信息需要在模型的多个部分表示
#
# --gradient_accumulation_steps 16
#   梯度累积步数为16，相当于将有效批次大小扩大到16
#   在显存有限的情况下，通过累积梯度模拟更大批次训练，提高稳定性
#
# --eval_steps 50
#   每50步进行一次评估，可以及时监控模型性能变化
#   50步是一个平衡点，既不会太频繁消耗计算资源，也不会太少导致反馈不及时
#   对于小数据集和LoRA微调，50步大约相当于查看10-20%训练进度的性能
#
# --save_steps 50
#   每50步保存一次模型检查点，与评估步数保持一致便于管理
#   如果训练中断，可以从最近的检查点恢复
#
# --save_total_limit 2
#   最多保存2个检查点，超过会删除旧的，避免占用过多磁盘空间
#
# --logging_steps 5
#   每5步记录一次训练日志，包括损失值等指标，方便监控训练进度
#   日志频率高于评估频率，可以更细粒度地观察训练动态
#
# --max_length 2048
#   设置输入序列的最大长度为2048个token
#   这个长度足够处理大多数对话场景，同时不会占用过多显存
#
# --output_dir output
#   指定输出目录，模型、日志和检查点都会保存在这里
#
# --warmup_ratio 0.05
#   学习率预热比例为0.05，即前5%的训练步数会逐渐增加学习率
#   预热可以避免训练初期学习率过大导致的不稳定
#
# --dataloader_num_workers 4
#   数据加载器的工作进程数为4，加速数据预处理
#   多进程加载可以减少CPU成为训练瓶颈的可能性
#
# --use_liger_kernel true
#   启用liger内核优化，可以提高训练速度
#   liger是一个针对Transformer模型优化的CUDA内核
#
# --load_from_cache_file false
#   不使用缓存文件，每次都重新处理数据集
#   在数据集较小或经常变化的情况下，禁用缓存可以确保使用最新数据
#
# --loss_scale ignore_empty_think
#   损失计算方式，忽略empty_think标记的部分（思考过程）
#   对于自我认知训练，我们只关注最终回答的质量，不关注思考过程
#
# --model_author swift
#   设置模型作者为swift，用于自我认知训练
#   这会替换自我认知数据集中的作者占位符
#
# --model_name swift-robot
#   设置模型名称为swift-robot，用于自我认知训练
#   这会替换自我认知数据集中的模型名称占位符
```

### 5.3 推理

完成微调后，我们可以使用微调后的 LoRA 适配器（LoRA Adapters）进行推理。在 SWIFT 框架中，这可以通过 `swift infer` 命令轻松实现。您只需要指定原始的基础模型类型和训练得到的检查点目录（即 LoRA 权重所在的目录）。

**示例命令:**

假设您的训练输出目录是 `output/qwen2_5-0_5b-lora/v1.0.0/checkpoint-xxx` (其中 `checkpoint-xxx` 是训练过程中最新的检查点文件夹)。

```bash
CUDA_VISIBLE_DEVICES=0  swift infer  --adapters output/v0-20250725-005426/checkpoint-100 --stream true  --temperature 0  --max_new_tokens 512
```

运行上述命令后，SWIFT 会加载 Qwen2.5-0.5B 基础模型和训练好的 LoRA 权重，然后进入一个交互式推理模式。您可以输入问题，模型将给出回答。

**推理时的工作原理:**

在推理时，SWIFT 会：

1.  加载原始的 **Qwen2.5-0.5B 基础模型**（参数是冻结的）。
2.  加载训练好的 **LoRA 适配器权重**。
3.  在进行前向传播计算时，动态地将 LoRA 权重与基础模型的相应层相结合（$W\_{final} = W\_{original} + B A$），从而实现微调后的行为。

这种方式的优点是，您可以在不修改原始大模型文件的情况下，灵活地切换不同的 LoRA 适配器，以适应不同的任务。

### 5.4 合并 LoRA：什么是合并，为什么合并？

#### 什么是合并 LoRA？

**合并 LoRA (Merging LoRA)** 是指将训练好的 LoRA 适配器的权重（即矩阵 $A$ 和 $B$）与原始预训练模型的相应权重矩阵 $W\_0$ 进行数学上的相加操作，生成一个新的、完整的模型权重 $W\_{merged} = W\_0 + BA$。这个过程将 LoRA 引入的少量修改固化到了原始模型的参数中。

#### 为什么合并 LoRA？

合并 LoRA 主要有以下几个目的和好处：

1.  **简化部署**: 合并后的模型是一个独立的、完整的模型，它不再需要依赖原始基础模型和单独的 LoRA 适配器文件。这意味着在部署推理服务时，您只需加载一个模型文件，这大大简化了模型管理和加载流程。
2.  **潜在的推理效率提升**: 尽管 LoRA 在推理时也可以动态结合，但每次计算都需要执行 $W\_0 + BA$ 的操作。合并后，这个加法操作在导出时就已经完成，推理时直接使用 $W\_{merged}$，可能会在某些硬件和软件栈上带来微小的推理速度提升（尽管通常可以忽略不计）。
3.  **兼容性**: 某些下游应用或框架可能只支持加载完整的模型权重文件（如 `.safetensors` 或 `.bin`），而不直接支持加载额外的 LoRA 适配器。合并 LoRA 可以解决这种兼容性问题。
4.  **分发便利性**: 如果您需要将微调后的模型分享给他人，提供一个合并后的完整模型文件比提供基础模型和单独的 LoRA 适配器文件要方便得多。

### 5.5 合并后权重的完整模型推理

在合并 LoRA 权重之后，您将得到一个包含了微调结果的完整模型。这个模型不再需要单独加载 LoRA 适配器，因为它已经“内置”了微调后的知识。您可以使用标准的模型加载和推理流程来使用它。

在 SWIFT 中，您可以使用 `swift export` 命令来合并 LoRA 权重并导出模型。

**示例命令 (合并 LoRA):**

```bash
CUDA_VISIBLE_DEVICES=0 swift export \
    --model_type qwen2_5-0_5b \
    --ckpt_dir output/qwen2_5-0_5b-lora/v1.0.0/checkpoint-xxx \
    --merge_lora true \
    --output_dir output/qwen2_5-0_5b-merged # 指定合并后模型的输出目录
```

运行此命令后，在 `output/qwen2_5-0_5b-merged` 目录下会生成一个完整的 Qwen2.5-0.5B 模型，其中包含了您微调的自我认知能力。

**合并后完整模型的推理:**

一旦模型被合并并导出，它的推理方式就与加载任何其他预训练模型一样，不再需要 `ckpt_dir` 参数来指定 LoRA 适配器，而是直接指向合并后模型的目录。

```bash
CUDA_VISIBLE_DEVICES=0 swift infer --model output/v0-20250725-005426/checkpoint-100-merged 
```

现在，当您向这个合并后的模型提问关于其自身的问题时，它将能够按照您在微调时设定的“自我认知”信息进行回答。